## Importing Required Packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as snpd
pd.set_option('display.max_columns', 2000)

## Reading Training Dataset

In [2]:
in_master_dataset = pd.read_csv("Datasets/train Data.csv", header=0, low_memory=False,sep=",")

Dropping id and release as they have no relastionship with the dataset. 

In [3]:
in_master_dataset.drop("id",axis=1,inplace=True)
in_master_dataset.drop("release",axis=1,inplace=True)

In [4]:
in_col_list = in_master_dataset.columns
tot_rcnt,tot_ccnt = in_master_dataset.shape[0],in_master_dataset.shape[1]
print("Rows ",tot_rcnt)
print("Cols ",tot_ccnt)

Rows  14644
Cols  1377


In [5]:
in_master_dataset.describe(include='all')

n_0000      n_0001        n_0002      n_0003     n_0004  \
count   19.000000  760.000000  14000.000000  200.000000  99.000000   
unique        NaN         NaN           NaN         NaN        NaN   
top           NaN         NaN           NaN         NaN        NaN   
freq          NaN         NaN           NaN         NaN        NaN   
mean     0.388664    0.661216      0.040932    0.102976   0.345118   
std      0.277911    0.211124      0.022779    0.151838   0.214672   
min      0.000000    0.000000      0.000000    0.000000   0.000000   
25%      0.153846    0.452381      0.032245    0.011905   0.200000   
50%      0.307692    0.714286      0.041477    0.047619   0.266667   
75%      0.576923    0.857143      0.049058    0.123016   0.466667   
max      0.923077    1.000000      1.000000    1.000000   1.000000   

              n_0005       n_0006       n_0007    n_0008     n_0009  \
count   13994.000000  2517.000000  1802.000000  7.000000  64.000000   
unique           NaN          NaN          NaN       NaN        NaN   
top              NaN          NaN          NaN       NaN        NaN   
freq             NaN          NaN          NaN       NaN        NaN   
mean        0.396736     0.197298     0.011224  0.444444   0.135827   
std         0.085333     0.237687     0.034121  0.435087   0.213308   
min         0.000000     0.000000     0.000000  0.000000   0.000000   
25%         0.342105     0.000000     0.000000  0.055556   0.019710   
50%         0.394737     0.200000     0.000000  0.444444   0.049793   
75%         0.421053     0.400000     0.011905  0.777778   0.149378   
max         1.000000     1.000000     1.000000  1.000000   1.000000   

           n_0010    n_0011        n_0012      n_0013      n_0014  \
count   52.000000  3.000000  10167.000000  280.000000  537.000000   
unique        NaN       NaN           NaN         NaN         NaN   
top           NaN       NaN           NaN         NaN         NaN   
freq          NaN       NaN           NaN         NaN         NaN   
mean     0.371154  0.666667      0.174716    0.389286    0.110242   
std      0.174150  0.577350      0.095420    0.488461    0.169027   
min      0.100000  0.000000      0.000000    0.000000    0.000000   
25%      0.300000  0.500000      0.097561    0.000000    0.000000   
50%      0.300000  1.000000      0.170732    0.000000    0.000000   
75%      0.400000  1.000000      0.256098    1.000000    0.200000   
max      1.000000  1.000000      1.000000    1.000000    1.000000   

             n_0015      n_0016       n_0017    n_0018        n_0019  \
count   1955.000000  509.000000  1082.000000  4.000000  13787.000000   
unique          NaN         NaN          NaN       NaN           NaN   
top             NaN         NaN          NaN       NaN           NaN   
freq            NaN         NaN          NaN       NaN           NaN   
mean       0.034115    0.418421     0.450144  0.250000      0.020222   
std        0.064712    0.152054     0.148163  0.288675      0.072008   
min        0.000000    0.000000     0.000000  0.000000      0.000000   
25%        0.000000    0.357143     0.388889  0.000000      0.000000   
50%        0.014493    0.428571     0.444444  0.250000      0.000000   
75%        0.028986    0.476190     0.500000  0.500000      0.000000   
max        1.000000    1.000000     1.000000  0.500000      1.000000   

            n_0020     n_0021      n_0022     n_0023      n_0024       n_0025  \
count   729.000000  32.000000  627.000000  45.000000  108.000000  1048.000000   
unique         NaN        NaN         NaN        NaN         NaN          NaN   
top            NaN        NaN         NaN        NaN         NaN          NaN   
freq           NaN        NaN         NaN        NaN         NaN          NaN   
mean      0.451192   0.229167    0.028969   0.166667    0.263889     0.005225   
std       0.178752   0.298593    0.072754   0.337100    0.160618     0.024284   
min       0.000000   0.000000    0.000000   0.000000    

## Data Preprocessing

#### Dropping variables

Variables having more than 80% misisng data are dropped

In [6]:
# type(in_master_dataset.isna().sum())
# in_master_dataset.isna().sum() > tot_rcnt*0.8
# Negate a boolean value in series element wise using ~ operator
filtered_in_master_dataset = in_master_dataset.loc[:,~(in_master_dataset.isna().sum() > tot_rcnt*0.8)]
filtered_in_master_dataset.describe(include='all')

,n_0002,n_0005,n_0012,n_0019,n_0028,n_0034,n_0038,n_0039,n_0047,n_0048,n_0050,n_0051,n_0052,n_0059,n_0060,n_0061,n_0064,n_0066,n_0067,n_0071,n_0074,n_0075,n_0078,n_0083,n_0086,n_0091,n_0095,n_0099,n_0100,n_0102,n_0108,n_0109,n_0110,o_0120,o_0125,o_0129,o_0132,o_0141,o_0144,o_0147,o_0152,o_0153,o_0154,o_0157,o_0168,o_0175,o_0176,o_0179,o_0201,o_0202,o_0208,o_0217,o_0221,o_0223,o_0230,o_0231,o_0241,o_0248,o_0264,o_0265,o_0268,o_0270,o_0274,o_0276,o_0279,o_0285,o_0286,o_0301,o_0314,o_0315,o_0323,c_0332,c_0337,c_0339,c_0348,c_0351,c_0357,c_0361,c_0364,c_0368,c_0369,c_0376,c_0377,c_0384,c_0389,c_0391,c_0401,c_0403,c_0405,c_0409,c_0412,c_0419,c_0422,c_0423,c_0424,c_0426,c_0428,c_0430,c_0432,c_0434,c_0437,c_0438,c_0441,c_0443,c_0444,c_0449,c_0453,c_0454,c_0456,c_0461,c_0463,c_0466,c_0468,c_0473,c_0475,c_0489,c_0492,c_0500,c_0502,c_0516,c_0523,c_0526,c_0527,c_0543,c_0544,c_0550,c_0554,c_0560,c_0564,c_0572,c_0573,c_0576,c_0578,c_0580,c_0582,c_0585,c_0587,c_0590,c_0591,c_0593,c_0601,c_0609,c_0621,c_0622,c_0623,c_0629,c_0636,c_0638,c_0639,c_0653,c_0655,c_0657,c_0658,c_0659,c_0662,c_0665,c_0671,c_0685,c_0696,c_0699,c_0704,c_0707,c_0709,c_0717,c_0722,c_0724,c_0727,c_0728,c_0738,c_0749,c_0752,c_0755,c_0757,c_0758,c_0761,c_0762,c_0764,c_0765,c_0768,c_0770,c_0772,c_0775,c_0785,c_0787,c_0794,c_0797,c_0798,c_0799,c_0809,c_0810,c_0814,c_0819,c_0826,c_0829,c_0830,c_0833,c_0834,c_0835,c_0836,c_0838,c_0845,c_0847,c_0851,c_0858,c_0862,c_0868,c_0870,c_0877,c_0887,c_0890,c_0891,c_0897,c_0905,c_0910,c_0917,c_0920,c_0923,c_0926,c_0935,c_0939,c_0944,c_0946,c_0947,c_0949,c_0951,c_0952,c_0953,c_0956,c_0957,c_0965,c_0975,c_0977,c_0978,c_0980,c_0982,c_0983,c_0984,c_0989,c_0991,c_0996,c_0997,c_1003,c_1004,c_1011,c_1012,c_1021,c_1029,c_1033,c_1041,c_1043,c_1044,c_1045,c_1049,c_1052,c_1053,c_1055,c_1061,c_1062,c_1064,c_1065,c_1069,c_1075,c_1090,c_1092,c_1094,c_1095,c_1099,c_1101,c_1104,c_1109,c_1114,c_1116,c_1120,c_1121,c_1122,c_1130,c_1131,c_1137,c_1142,c_1144,c_1145,c_1149,c_1158,c_1164,c_1169,c_1171,c_1172,c_1175,c_1183,c_1186,c_1189,c_1190,c_1192,c_1198,c_1201,c_1206,c_1210,c_1212,c_1213,c_1217,c_1218,c_1223,c_1225,c_1227,c_1231,c_1234,c_1235,c_1236,c_1237,c_1238,c_1239,c_1243,c_1244,c_1247,c_1248,c_1252,c_1259,c_1267,c_1270,c_1282,c_1286,c_1297,c_1302,c_1309,c_1316,c_1320,c_1326,c_1328,c_1330,c_1333,c_1335,c_1343,c_1347,c_1348,c_1361,c_1363,c_1372,c_1374,c_1377
count,14000.000000,13994.000000,10167.000000,13787.000000,3006.000000,7512.000000,13303.000000,4057.000000,14644.0,3673.000000,14644.0,3048.000000,14644.0,3152.000000,3154.000000,14644.0,8619.000000,3002.000000,14644.000000,4190.000000,4097.000000,14644.0,14644.000000,14630.000000,10184.000000,14644.0,3155.000000,7606.000000,7438.000000,9981.000000,14644.000000,14644.000000,12754.000000,14605.000000,7371.000000,3024.000000,4864.000000,9596.000000,14589.000000,5227.000000,3174.000000,12224.000000,3175.000000,3176.000000,7135.000000,8430.000000,14644.000000,3473.000000,14416.000000,4782.000000,3004.000000,7486.000000,4130.000000,7519.000000,14519.000000,3025.000000,7453.000000,3013.000000,14644.000000,6579.000000,7002.000000,7613.000000,4811.000000,3229.000000,8483.000000,3817.000000,4916.000000,9136.000000,3176.000000,7345.000000,3175.000000,3166,10085,3171,8834,8932,4484,6827,10109,14633,6884,2981,5078,7539,9216,7279,14424,3576,8794,4792,3648,3183,3148,3168,3172,13710,3167,3010,6798,9449,9050,6508,6824,5463,14543,7534,10091,10113,14455,14625,4111,14644,10115,3165,8425,3185,4924,14644,6814,6372,9363,3185,10084,13926,14597,11954,13989,5338,11122,3502,7372,5332,3172,5481,14637,10088,9135,14491,10167,10155,14343,10105,11176,6433,14335,3179,10110,14644,4087,14629,10107,14634,3134,10085,14462,13940,7439,4078,8904,14644,14326,14278,14637,7532,10084,3028,8713,9184,14644,10087,4876,14299,10090,3185,14644,14034,13783,3136,10085,14644,2993,12078,11958,4348,11475,3154,10167,5930,13929,3577,8746,10083,10091,2982,3996,8868,9817,9769,10124,14644,14636,10086,3158,14369,3154,3154,14644,4092,13934,8583,14629,7769,100

In [7]:
filtered_in_master_dataset.head()

,n_0002,n_0005,n_0012,n_0019,n_0028,n_0034,n_0038,n_0039,n_0047,n_0048,n_0050,n_0051,n_0052,n_0059,n_0060,n_0061,n_0064,n_0066,n_0067,n_0071,n_0074,n_0075,n_0078,n_0083,n_0086,n_0091,n_0095,n_0099,n_0100,n_0102,n_0108,n_0109,n_0110,o_0120,o_0125,o_0129,o_0132,o_0141,o_0144,o_0147,o_0152,o_0153,o_0154,o_0157,o_0168,o_0175,o_0176,o_0179,o_0201,o_0202,o_0208,o_0217,o_0221,o_0223,o_0230,o_0231,o_0241,o_0248,o_0264,o_0265,o_0268,o_0270,o_0274,o_0276,o_0279,o_0285,o_0286,o_0301,o_0314,o_0315,o_0323,c_0332,c_0337,c_0339,c_0348,c_0351,c_0357,c_0361,c_0364,c_0368,c_0369,c_0376,c_0377,c_0384,c_0389,c_0391,c_0401,c_0403,c_0405,c_0409,c_0412,c_0419,c_0422,c_0423,c_0424,c_0426,c_0428,c_0430,c_0432,c_0434,c_0437,c_0438,c_0441,c_0443,c_0444,c_0449,c_0453,c_0454,c_0456,c_0461,c_0463,c_0466,c_0468,c_0473,c_0475,c_0489,c_0492,c_0500,c_0502,c_0516,c_0523,c_0526,c_0527,c_0543,c_0544,c_0550,c_0554,c_0560,c_0564,c_0572,c_0573,c_0576,c_0578,c_0580,c_0582,c_0585,c_0587,c_0590,c_0591,c_0593,c_0601,c_0609,c_0621,c_0622,c_0623,c_0629,c_0636,c_0638,c_0639,c_0653,c_0655,c_0657,c_0658,c_0659,c_0662,c_0665,c_0671,c_0685,c_0696,c_0699,c_0704,c_0707,c_0709,c_0717,c_0722,c_0724,c_0727,c_0728,c_0738,c_0749,c_0752,c_0755,c_0757,c_0758,c_0761,c_0762,c_0764,c_0765,c_0768,c_0770,c_0772,c_0775,c_0785,c_0787,c_0794,c_0797,c_0798,c_0799,c_0809,c_0810,c_0814,c_0819,c_0826,c_0829,c_0830,c_0833,c_0834,c_0835,c_0836,c_0838,c_0845,c_0847,c_0851,c_0858,c_0862,c_0868,c_0870,c_0877,c_0887,c_0890,c_0891,c_0897,c_0905,c_0910,c_0917,c_0920,c_0923,c_0926,c_0935,c_0939,c_0944,c_0946,c_0947,c_0949,c_0951,c_0952,c_0953,c_0956,c_0957,c_0965,c_0975,c_0977,c_0978,c_0980,c_0982,c_0983,c_0984,c_0989,c_0991,c_0996,c_0997,c_1003,c_1004,c_1011,c_1012,c_1021,c_1029,c_1033,c_1041,c_1043,c_1044,c_1045,c_1049,c_1052,c_1053,c_1055,c_1061,c_1062,c_1064,c_1065,c_1069,c_1075,c_1090,c_1092,c_1094,c_1095,c_1099,c_1101,c_1104,c_1109,c_1114,c_1116,c_1120,c_1121,c_1122,c_1130,c_1131,c_1137,c_1142,c_1144,c_1145,c_1149,c_1158,c_1164,c_1169,c_1171,c_1172,c_1175,c_1183,c_1186,c_1189,c_1190,c_1192,c_1198,c_1201,c_1206,c_1210,c_1212,c_1213,c_1217,c_1218,c_1223,c_1225,c_1227,c_1231,c_1234,c_1235,c_1236,c_1237,c_1238,c_1239,c_1243,c_1244,c_1247,c_1248,c_1252,c_1259,c_1267,c_1270,c_1282,c_1286,c_1297,c_1302,c_1309,c_1316,c_1320,c_1326,c_1328,c_1330,c_1333,c_1335,c_1343,c_1347,c_1348,c_1361,c_1363,c_1372,c_1374,c_1377
0,0.025449,0.368421,0.292683,0.0,NaN,0.223881,0.193548,NaN,1,NaN,1,NaN,1,NaN,NaN,1,0.00,NaN,0.928571,NaN,NaN,1,0.800000,1.000000,0.450,1,NaN,NaN,0.12,0.242424,0.800000,0.1875,0.009231,0.0,6.0,NaN,9.0,15.0,8.0,4.0,NaN,4.0,NaN,NaN,12.0,10.0,303,0.0,0.0,2.0,NaN,5.0,24.0,10.0,1.0,NaN,19.0,NaN,7,NaN,NaN,NaN,27.0,NaN,10.0,NaN,15.0,12.0,NaN,0.0,NaN,NaN,a,NaN,f,o,NaN,NaN,a,b,f,NaN,NaN,a,b,g,a,NaN,e,a,NaN,NaN,NaN,NaN,NaN,a,NaN,NaN,b,f,n,a,NaN,c,a,a,b,b,a,a,b,a,a,NaN,NaN,NaN,b,a,NaN,NaN,i,NaN,a,b,a,a,a,b,a,b,d,r,NaN,a,a,a,u,e,g,e,b,a,b,NaN,a,NaN,b,d,NaN,c,b,d,NaN,b,a,a,d,NaN,e,b,b,b,b,a,b,NaN,f,u,a,m,a,b,a,NaN,c,a,c,NaN,a,c,NaN,d,c,NaN,a,NaN,b,NaN,b,NaN,v,b,a,NaN,NaN,t,a,v,b,a,d,b,NaN,b,NaN,NaN,b,NaN,a,f,e,b,a,b,h,r,a,NaN,NaN,a,i,u,a,NaN,a,a,NaN,b,c,h,a,NaN,NaN,c,NaN,b,NaN,b,b,a,c,b,b,a,a,j,s,h,NaN,f,NaN,h,b,a,NaN,a,NaN,l,NaN,d,a,a,NaN,NaN,g,v,d,d,v,x,NaN,NaN,a,b,a,c,b,c,NaN,c,b,m,g,e,e,u,s,p,NaN,a,b,s,b,a,a,NaN,c,b,NaN,b,NaN,c,n,a,NaN,a,NaN,c,b,a,v,NaN,d,o,NaN,b,n,b,b,b,b,NaN,NaN,NaN,b,NaN,g,b,a,e,w,b,NaN,b,e,b,a,q,NaN
1,0.031297,0.315789,0.243902,0.0,NaN,0.104478,0.177419,NaN,1,NaN,1,NaN,1,NaN,NaN,1,0.00,NaN,0.928571,NaN,NaN,1,0.666667,0.000000,0.475,1,NaN,NaN,0.18,0.484848,0.666667,0.1875,0.009231,0.0,0.0,NaN,6.0,19.0,8.0,NaN,NaN,5.0,NaN,NaN,19.0,9.0,293,11.0,2.0,11.0,NaN,5.0,25.0,16.0,2.0,NaN,21.0,NaN,7,2.0,8.0,6.0,33.0,NaN,14.0,NaN,14.0,10.0,NaN,0.0,NaN,NaN,a,NaN,k,s,NaN,NaN,a,b,e,NaN,NaN,a,NaN,NaN,a,NaN,g,NaN,NaN,NaN,NaN,NaN,NaN,b,NaN,NaN,NaN,m,g,NaN,NaN,NaN,a,a,b,b,a,a,a,a,a,NaN,d,NaN,NaN,h,NaN,NaN,e,NaN,a,b,e,b,a,b,a,NaN,c,NaN,NaN,NaN,f,a,l,d,k,i,b,a,b,NaN,b,NaN,b,d,NaN,c,b,b,NaN,b,b,b,d,NaN,k,b,a,a,a

Variables with single unique value are dropped

In [8]:
type(filtered_in_master_dataset.describe(include='all'))

pandas.core.frame.DataFrame

In [9]:
in_master_dataset_summary = pd.DataFrame(filtered_in_master_dataset.describe(include='all'))

In [10]:
filtered_in_master_dataset = filtered_in_master_dataset.loc[:,~(in_master_dataset_summary.loc["min"] 
                                                                == in_master_dataset_summary.loc["max"])]

In [11]:
filtered_in_master_dataset = filtered_in_master_dataset.loc[:,~(in_master_dataset_summary.loc["unique"]==1)]

In [12]:
filtered_in_master_dataset.describe(include='all')

,n_0002,n_0005,n_0012,n_0019,n_0028,n_0034,n_0038,n_0039,n_0048,n_0051,n_0059,n_0060,n_0064,n_0066,n_0067,n_0071,n_0074,n_0078,n_0083,n_0086,n_0095,n_0099,n_0100,n_0102,n_0108,n_0109,n_0110,o_0120,o_0125,o_0129,o_0132,o_0141,o_0144,o_0147,o_0152,o_0153,o_0154,o_0157,o_0168,o_0175,o_0176,o_0179,o_0201,o_0202,o_0208,o_0217,o_0221,o_0223,o_0230,o_0231,o_0241,o_0248,o_0264,o_0265,o_0268,o_0270,o_0274,o_0276,o_0279,o_0285,o_0286,o_0301,o_0314,o_0315,o_0323,c_0332,c_0337,c_0339,c_0348,c_0351,c_0357,c_0361,c_0364,c_0368,c_0369,c_0376,c_0377,c_0384,c_0389,c_0391,c_0401,c_0403,c_0405,c_0409,c_0412,c_0419,c_0422,c_0423,c_0424,c_0426,c_0428,c_0430,c_0432,c_0434,c_0437,c_0438,c_0441,c_0443,c_0444,c_0449,c_0453,c_0454,c_0456,c_0461,c_0463,c_0466,c_0468,c_0473,c_0475,c_0489,c_0492,c_0500,c_0502,c_0516,c_0523,c_0526,c_0527,c_0543,c_0544,c_0550,c_0554,c_0560,c_0564,c_0572,c_0573,c_0576,c_0578,c_0580,c_0582,c_0585,c_0587,c_0590,c_0591,c_0593,c_0601,c_0609,c_0621,c_0622,c_0623,c_0629,c_0636,c_0638,c_0639,c_0653,c_0655,c_0657,c_0658,c_0659,c_0662,c_0665,c_0671,c_0685,c_0696,c_0699,c_0704,c_0707,c_0709,c_0717,c_0722,c_0724,c_0727,c_0728,c_0738,c_0749,c_0752,c_0755,c_0757,c_0758,c_0761,c_0762,c_0764,c_0765,c_0768,c_0770,c_0772,c_0775,c_0785,c_0787,c_0794,c_0797,c_0798,c_0799,c_0809,c_0810,c_0814,c_0819,c_0826,c_0829,c_0830,c_0833,c_0834,c_0835,c_0836,c_0838,c_0845,c_0847,c_0851,c_0858,c_0862,c_0868,c_0870,c_0877,c_0887,c_0890,c_0891,c_0897,c_0905,c_0910,c_0917,c_0920,c_0923,c_0926,c_0935,c_0939,c_0944,c_0946,c_0947,c_0949,c_0951,c_0952,c_0953,c_0956,c_0957,c_0965,c_0975,c_0977,c_0978,c_0980,c_0982,c_0983,c_0984,c_0989,c_0991,c_0996,c_0997,c_1003,c_1004,c_1011,c_1012,c_1021,c_1029,c_1033,c_1041,c_1043,c_1044,c_1045,c_1049,c_1052,c_1053,c_1055,c_1061,c_1062,c_1064,c_1065,c_1069,c_1075,c_1090,c_1092,c_1094,c_1095,c_1099,c_1101,c_1104,c_1109,c_1114,c_1116,c_1120,c_1121,c_1122,c_1130,c_1131,c_1137,c_1142,c_1144,c_1145,c_1149,c_1158,c_1164,c_1169,c_1171,c_1172,c_1175,c_1183,c_1186,c_1189,c_1190,c_1192,c_1198,c_1201,c_1206,c_1210,c_1212,c_1213,c_1217,c_1218,c_1223,c_1225,c_1227,c_1231,c_1234,c_1235,c_1236,c_1237,c_1238,c_1239,c_1243,c_1244,c_1247,c_1248,c_1252,c_1259,c_1267,c_1270,c_1282,c_1286,c_1297,c_1302,c_1309,c_1316,c_1320,c_1326,c_1328,c_1330,c_1333,c_1335,c_1343,c_1347,c_1348,c_1361,c_1363,c_1372,c_1374,c_1377
count,14000.000000,13994.000000,10167.000000,13787.000000,3006.000000,7512.000000,13303.000000,4057.000000,3673.000000,3048.000000,3152.000000,3154.000000,8619.000000,3002.000000,14644.000000,4190.000000,4097.000000,14644.000000,14630.000000,10184.000000,3155.000000,7606.000000,7438.000000,9981.000000,14644.000000,14644.000000,12754.000000,14605.000000,7371.000000,3024.000000,4864.000000,9596.000000,14589.000000,5227.000000,3174.000000,12224.000000,3175.000000,3176.000000,7135.000000,8430.000000,14644.000000,3473.000000,14416.000000,4782.000000,3004.000000,7486.000000,4130.000000,7519.000000,14519.000000,3025.000000,7453.000000,3013.000000,14644.000000,6579.000000,7002.000000,7613.000000,4811.000000,3229.000000,8483.000000,3817.000000,4916.000000,9136.000000,3176.000000,7345.000000,3175.000000,3166,10085,3171,8834,8932,4484,6827,10109,14633,6884,2981,5078,7539,9216,7279,14424,3576,8794,4792,3648,3183,3148,3168,3172,13710,3167,3010,6798,9449,9050,6508,6824,5463,14543,7534,10091,10113,14455,14625,4111,14644,10115,3165,8425,3185,4924,14644,6814,6372,9363,3185,10084,13926,14597,11954,13989,5338,11122,3502,7372,5332,3172,5481,14637,10088,9135,14491,10167,10155,14343,10105,11176,6433,14335,3179,10110,14644,4087,14629,10107,14634,3134,10085,14462,13940,7439,4078,8904,14644,14326,14278,14637,7532,10084,3028,8713,9184,14644,10087,4876,14299,10090,3185,14644,14034,13783,3136,10085,14644,2993,12078,11958,4348,11475,3154,10167,5930,13929,3577,8746,10083,10091,2982,3996,8868,9817,9769,10124,14644,14636,10086,3158,14369,3154,3154,14644,4092,13934,8583,14629,7769,10091,10164,13957,9389,7352,3136,6364,14440,9672,8676,10113,3050,8245,12456,3135,13983,10167,

In [13]:
filtered_in_master_dataset.shape

(14644, 333)

#### Splitting datasets based on type

In [14]:
num_in_master_dataset = filtered_in_master_dataset.filter(like="n", axis=1).copy()
ord_in_master_dataset = filtered_in_master_dataset.filter(like="o", axis=1).copy()
cat_in_master_dataset = filtered_in_master_dataset.filter(like="c", axis=1).copy()

num_in_master_dataset.shape[1]+ord_in_master_dataset.shape[1]+cat_in_master_dataset.shape[1]

333

### Handling Missing Data

#### Categorical Data

Label Encoding

In [15]:
cat_in_master_dataset.describe(include="all")

,c_0332,c_0337,c_0339,c_0348,c_0351,c_0357,c_0361,c_0364,c_0368,c_0369,c_0376,c_0377,c_0384,c_0389,c_0391,c_0401,c_0403,c_0405,c_0409,c_0412,c_0419,c_0422,c_0423,c_0424,c_0426,c_0428,c_0430,c_0432,c_0434,c_0437,c_0438,c_0441,c_0443,c_0444,c_0449,c_0453,c_0454,c_0456,c_0461,c_0463,c_0466,c_0468,c_0473,c_0475,c_0489,c_0492,c_0500,c_0502,c_0516,c_0523,c_0526,c_0527,c_0543,c_0544,c_0550,c_0554,c_0560,c_0564,c_0572,c_0573,c_0576,c_0578,c_0580,c_0582,c_0585,c_0587,c_0590,c_0591,c_0593,c_0601,c_0609,c_0621,c_0622,c_0623,c_0629,c_0636,c_0638,c_0639,c_0653,c_0655,c_0657,c_0658,c_0659,c_0662,c_0665,c_0671,c_0685,c_0696,c_0699,c_0704,c_0707,c_0709,c_0717,c_0722,c_0724,c_0727,c_0728,c_0738,c_0749,c_0752,c_0755,c_0757,c_0758,c_0761,c_0762,c_0764,c_0765,c_0768,c_0770,c_0772,c_0775,c_0785,c_0787,c_0794,c_0797,c_0798,c_0799,c_0809,c_0810,c_0814,c_0819,c_0826,c_0829,c_0830,c_0833,c_0834,c_0835,c_0836,c_0838,c_0845,c_0847,c_0851,c_0858,c_0862,c_0868,c_0870,c_0877,c_0887,c_0890,c_0891,c_0897,c_0905,c_0910,c_0917,c_0920,c_0923,c_0926,c_0935,c_0939,c_0944,c_0946,c_0947,c_0949,c_0951,c_0952,c_0953,c_0956,c_0957,c_0965,c_0975,c_0977,c_0978,c_0980,c_0982,c_0983,c_0984,c_0989,c_0991,c_0996,c_0997,c_1003,c_1004,c_1011,c_1012,c_1021,c_1029,c_1033,c_1041,c_1043,c_1044,c_1045,c_1049,c_1052,c_1053,c_1055,c_1061,c_1062,c_1064,c_1065,c_1069,c_1075,c_1090,c_1092,c_1094,c_1095,c_1099,c_1101,c_1104,c_1109,c_1114,c_1116,c_1120,c_1121,c_1122,c_1130,c_1131,c_1137,c_1142,c_1144,c_1145,c_1149,c_1158,c_1164,c_1169,c_1171,c_1172,c_1175,c_1183,c_1186,c_1189,c_1190,c_1192,c_1198,c_1201,c_1206,c_1210,c_1212,c_1213,c_1217,c_1218,c_1223,c_1225,c_1227,c_1231,c_1234,c_1235,c_1236,c_1237,c_1238,c_1239,c_1243,c_1244,c_1247,c_1248,c_1252,c_1259,c_1267,c_1270,c_1282,c_1286,c_1297,c_1302,c_1309,c_1316,c_1320,c_1326,c_1328,c_1330,c_1333,c_1335,c_1343,c_1347,c_1348,c_1361,c_1363,c_1372,c_1374,c_1377
count,3166,10085,3171,8834,8932,4484,6827,10109,14633,6884,2981,5078,7539,9216,7279,14424,3576,8794,4792,3648,3183,3148,3168,3172,13710,3167,3010,6798,9449,9050,6508,6824,5463,14543,7534,10091,10113,14455,14625,4111,14644,10115,3165,8425,3185,4924,14644,6814,6372,9363,3185,10084,13926,14597,11954,13989,5338,11122,3502,7372,5332,3172,5481,14637,10088,9135,14491,10167,10155,14343,10105,11176,6433,14335,3179,10110,14644,4087,14629,10107,14634,3134,10085,14462,13940,7439,4078,8904,14644,14326,14278,14637,7532,10084,3028,8713,9184,14644,10087,4876,14299,10090,3185,14644,14034,13783,3136,10085,14644,2993,12078,11958,4348,11475,3154,10167,5930,13929,3577,8746,10083,10091,2982,3996,8868,9817,9769,10124,14644,14636,10086,3158,14369,3154,3154,14644,4092,13934,8583,14629,7769,10091,10164,13957,9389,7352,3136,6364,14440,9672,8676,10113,3050,8245,12456,3135,13983,10167,9420,14630,3507,5689,14644,3148,13988,3159,4108,10084,14640,5242,9622,14634,10084,10089,9025,9791,8917,3894,9258,3185,9536,10972,14237,3133,14457,2981,9501,3185,11815,14383,14413,3172,6369,8634,6330,9292,14609,9332,9591,3185,3115,10085,10093,13984,14636,14444,5115,4484,10167,14644,9225,14644,10134,8997,8611,7976,10084,3184,14424,14644,9714,10095,10089,10114,3181,12981,10083,6081,10110,3176,14644,9741,14644,2942,10090,4427,14617,7505,10951,9473,3122,14644,9649,3183,14395,14644,10103,10116,9127,14637,3185,3161,3995,14464,5086,9088,10081,10090,7472,8964,10114,6823,14642,10202,10090,14640,5792,3573
unique,2,2,2,23,22,3,2,2,2,22,2,2,2,2,23,2,8,22,23,5,2,2,6,6,2,2,3,6,23,23,2,5,3,2,2,2,2,2,2,4,3,2,2,5,2,2,9,6,3,23,2,2,3,5,3,2,2,6,7,4,22,2,2,6,2,24,4,16,12,2,2,2,2,2,5,2,5,2,5,2,6,2,2,2,2,6,2,23,3,2,2,2,2,2,11,22,24,2,14,2,2,2,2,3,3,3,2,2,4,2,5,3,5,2,4,3,11,2,2,22,2,2,2,2,23,2,22,2,2,6,2,2,2,3,2,2,3,2,22,5,5,2,2,8,22,3,2,7,2,24,22,2,2,2,2,2,2,7,23,3,5,5,4,2,2,2,5,2,2,4,24,2,2,2,23,22,22,21,22,3,23,2,2,2,2,19,23,2,4,2,2,2,4,22,23,23,5,22,23,2,2,2,2,2,3,2,3,2,4,3,24,11,5,23,22,23,20,2,2,2,24,2,2,2,4,10,2,5,2,2,4,23,2,23,2,5,3,2,4,23,4,6,23,2,2,32,2,2,3,2,2,4,5,2,2,23,2,2,6,22,2,6,4,8,2,2,24,2
top,a,a,b,p,f,c,a,a,b,l,b,a,a,b,q,a,a,l,f,e,b,b,f,c,

In [16]:
cat_in_dict = defaultdict(LabelEncoder)
for i in cat_in_master_dataset.columns:
    cat_in_master_dataset[i].fillna("None", inplace=True)
    cat_in_master_dataset[i] = cat_in_master_dataset[i].astype("category")
    cat_in_dict[i] = LabelEncoder()
    cat_in_master_dataset[i] = cat_in_dict[i].fit_transform(cat_in_master_dataset[i])

In [17]:
cat_in_master_dataset.describe(include="all")

,c_0332,c_0337,c_0339,c_0348,c_0351,c_0357,c_0361,c_0364,c_0368,c_0369,c_0376,c_0377,c_0384,c_0389,c_0391,c_0401,c_0403,c_0405,c_0409,c_0412,c_0419,c_0422,c_0423,c_0424,c_0426,c_0428,c_0430,c_0432,c_0434,c_0437,c_0438,c_0441,c_0443,c_0444,c_0449,c_0453,c_0454,c_0456,c_0461,c_0463,c_0466,c_0468,c_0473,c_0475,c_0489,c_0492,c_0500,c_0502,c_0516,c_0523,c_0526,c_0527,c_0543,c_0544,c_0550,c_0554,c_0560,c_0564,c_0572,c_0573,c_0576,c_0578,c_0580,c_0582,c_0585,c_0587,c_0590,c_0591,c_0593,c_0601,c_0609,c_0621,c_0622,c_0623,c_0629,c_0636,c_0638,c_0639,c_0653,c_0655,c_0657,c_0658,c_0659,c_0662,c_0665,c_0671,c_0685,c_0696,c_0699,c_0704,c_0707,c_0709,c_0717,c_0722,c_0724,c_0727,c_0728,c_0738,c_0749,c_0752,c_0755,c_0757,c_0758,c_0761,c_0762,c_0764,c_0765,c_0768,c_0770,c_0772,c_0775,c_0785,c_0787,c_0794,c_0797,c_0798,c_0799,c_0809,c_0810,c_0814,c_0819,c_0826,c_0829,c_0830,c_0833,c_0834,c_0835,c_0836,c_0838,c_0845,c_0847,c_0851,c_0858,c_0862,c_0868,c_0870,c_0877,c_0887,c_0890,c_0891,c_0897,c_0905,c_0910,c_0917,c_0920,c_0923,c_0926,c_0935,c_0939,c_0944,c_0946,c_0947,c_0949,c_0951,c_0952,c_0953,c_0956,c_0957,c_0965,c_0975,c_0977,c_0978,c_0980,c_0982,c_0983,c_0984,c_0989,c_0991,c_0996,c_0997,c_1003,c_1004,c_1011,c_1012,c_1021,c_1029,c_1033,c_1041,c_1043,c_1044,c_1045,c_1049,c_1052,c_1053,c_1055,c_1061,c_1062,c_1064,c_1065,c_1069,c_1075,c_1090,c_1092,c_1094,c_1095,c_1099,c_1101,c_1104,c_1109,c_1114,c_1116,c_1120,c_1121,c_1122,c_1130,c_1131,c_1137,c_1142,c_1144,c_1145,c_1149,c_1158,c_1164,c_1169,c_1171,c_1172,c_1175,c_1183,c_1186,c_1189,c_1190,c_1192,c_1198,c_1201,c_1206,c_1210,c_1212,c_1213,c_1217,c_1218,c_1223,c_1225,c_1227,c_1231,c_1234,c_1235,c_1236,c_1237,c_1238,c_1239,c_1243,c_1244,c_1247,c_1248,c_1252,c_1259,c_1267,c_1270,c_1282,c_1286,c_1297,c_1302,c_1309,c_1316,c_1320,c_1326,c_1328,c_1330,c_1333,c_1335,c_1343,c_1347,c_1348,c_1361,c_1363,c_1372,c_1374,c_1377
count,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.00000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000,14644.000000

One Hot Encoding

In [18]:
cat_le_in_dict = defaultdict(OneHotEncoder)
for i in cat_in_master_dataset.columns:
    cat_le_in_dict[i] = OneHotEncoder()
    temp = cat_le_in_dict[i].fit_transform(cat_in_master_dataset[[i]])
    temp_= pd.DataFrame(temp.toarray(),columns=[(i+"_"+str(j)) for j in cat_in_master_dataset[i].value_counts().index])
    temp=temp_.set_index(cat_in_master_dataset.index.values)
    cat_in_master_dataset=pd.concat([cat_in_master_dataset,temp],axis=1)
    cat_in_master_dataset.drop(i,axis=1,inplace=True)

In [23]:
cat_in_master_dataset.head()

c_0332_0  c_0332_1  c_0332_2  c_0337_1  c_0337_0  c_0337_2  c_0339_0  \
0       1.0       0.0       0.0       0.0       1.0       0.0       1.0   
1       1.0       0.0       0.0       0.0       1.0       0.0       1.0   
2       1.0       0.0       0.0       0.0       0.0       1.0       1.0   
3       1.0       0.0       0.0       0.0       1.0       0.0       1.0   
4       0.0       1.0       0.0       0.0       1.0       0.0       0.0   

   c_0339_2  c_0339_1  c_0348_0  c_0348_16  c_0348_6  c_0348_8  c_0348_11  \
0       0.0       0.0       0.0        0.0       0.0       0.0        0.0   
1       0.0       0.0       0.0        0.0       0.0       0.0        0.0   
2       0.0       0.0       0.0        0.0       0.0       0.0        0.0   
3       0.0       0.0       0.0        0.0       0.0       0.0        0.0   
4       0.0       1.0       0.0        0.0       0.0       0.0        0.0   

   c_0348_18  c_0348_20  c_0348_17  c_0348_10  c_0348_14  c_0348_19  c_0348_2  \
0        0.0        1.0        0.0        0.0        0.0        0.0       0.0   
1        0.0        0.0        0.0        0.0        0.0        0.0       1.0   
2        0.0        0.0        0.0        0.0        0.0        0.0       0.0   
3        0.0        1.0        0.0        0.0        0.0        0.0       0.0   
4        0.0        0.0        0.0        0.0        0.0        0.0       0.0   

   c_0348_1  c_0348_5  c_0348_23  c_0348_13  c_0348_4  c_0348_3  c_0348_15  \
0       0.0       0.0        0.0        0.0       0.0       0.0        0.0   
1       0.0       0.0        0.0        0.0       0.0       0.0        0.0   
2       0.0       0.0        0.0        0.0       1.0       0.0        0.0   
3       0.0       0.0        0.0        0.0       0.0       0.0        0.0   
4       0.0       0.0        0.0        0.0       0.0       0.0        0.0   

   c_0348_22  c_0348_9  c_0348_12  c_0348_21  c_0348_7  c_0351_0  c_0351_5  \
0        0.0       0.0        0.0        0.0       0.0       0.0       0.0   
1        0.0       0.0        0.0        0.0       0.0       0.0       0.0   
2        0.0       0.0        0.0        0.0       0.0       0.0       0.0   
3        0.0       0.0        0.0        0.0       0.0       0.0       0.0   
4        0.0       1.0        0.0        0.0       0.0       0.0       0.0   

   c_0351_14  c_0351_12  c_0351_18  c_0351_20  c_0351_2  c_0351_11  c_0351_8  \
0        0.0        0.0        0.0        0.0       0.0        0.0       0.0   
1        0.0        0.0        0.0        0.0       0.0        0.0       0.0   
2        0.0        0.0        0.0        1.0       0.0        0.0       0.0   
3        0.0        0.0        0.0        0.0       0.0        0.0       0.0   
4        1.0        0.0        0.0        0.0       0.0        0.0       0.0   

   c_0351_19  c_0351_6  c_0351_15  c_0351_9  c_0351_22  c_0351_17  c_0351_10  \
0        0.0       0.0        0.0       0.0        0.0        1.0        0.0   
1        0.0       0.0        0.0       0.0        0.0        0.0        0.0   
2        0.0       0.0        0.0       0.0        0.0        0.0        0.0   
3        0.0       0.0        0.0       0.0        0.0        1.0        0.0   
4        0.0       0.0        0.0       0.0        0.0        0.0        0.0   

   c_0351_4  c_0351_16  c_0351_3  c_0351_1  c_0351_13  c_0351_21  c_0351_7  \
0       0.0        0.0       0.0       0.0        0.0        0.0       0.0   
1       0.0        0.0       1.0       0.0        0.0        0.0       0.0   
2       0.0        0.0       0.0       0.0        0.0        0.0       0.0   
3       0.0        0.0       0.0       0.0        0.0        0.0       0.0   
4       0.0        0.0       0.0       0.0        0.0        0.0       0.0   

   c_0357_0  c_0357_3  c_0357_2  c_0357_1  c_0361_0  c_0361_1  c_0361_2  \
0       1.0       0.0       0.0       0.0       1.0       0.0       0.0   
1       1.0       0.0       0.0       0.0       1.0       0.0       0.0   
2       

#### Ordinal Data

In [20]:
ord_in_master_dataset.describe(include="all")

,o_0120,o_0125,o_0129,o_0132,o_0141,o_0144,o_0147,o_0152,o_0153,o_0154,o_0157,o_0168,o_0175,o_0176,o_0179,o_0201,o_0202,o_0208,o_0217,o_0221,o_0223,o_0230,o_0231,o_0241,o_0248,o_0264,o_0265,o_0268,o_0270,o_0274,o_0276,o_0279,o_0285,o_0286,o_0301,o_0314,o_0315,o_0323
count,14605.000000,7371.000000,3024.000000,4864.000000,9596.000000,14589.000000,5227.000000,3174.000000,12224.000000,3175.000000,3176.000000,7135.000000,8430.000000,14644.000000,3473.000000,14416.000000,4782.000000,3004.000000,7486.000000,4130.000000,7519.000000,14519.000000,3025.000000,7453.000000,3013.000000,14644.000000,6579.000000,7002.000000,7613.000000,4811.000000,3229.000000,8483.000000,3817.000000,4916.000000,9136.000000,3176.000000,7345.000000,3175.000000
mean,0.475180,5.246914,0.303241,5.847862,28.539808,7.467338,2.293667,0.473535,2.423757,0.020157,0.006297,25.923055,8.736062,199.899071,5.077455,1.728496,5.526767,6.269973,4.721480,22.269492,22.968214,1.372960,22.089256,22.047498,23.659144,4.591027,5.545676,5.569123,5.506633,36.141135,1.855373,27.533538,1.708672,22.146054,5.571585,0.002834,0.087815,0.011969
std,0.904213,3.417582,0.747458,3.147397,7.921026,1.672322,1.609281,1.220075,1.593956,0.185068,0.130262,6.101340,1.632935,118.534900,3.696436,1.126582,3.418152,1.772628,1.159912,5.270109,7.834221,1.047416,6.516156,2.572596,6.210317,2.237666,3.443878,3.402803,3.223194,8.180413,1.375509,8.266226,1.501401,7.003053,3.875020,0.073118,0.323877,0.164170
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,4.000000,23.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000,23.000000,8.000000,94.000000,2.000000,1.000000,3.000000,5.000000,4.000000,20.000000,17.000000,1.000000,18.000000,20.000000,20.000000,4.000000,3.000000,3.000000,3.000000,30.000000,1.000000,22.000000,1.000000,17.000000,2.000000,0.000000,0.000000,0.000000
50%,0.000000,5.000000,0.000000,6.000000,29.000000,7.000000,2.000000,0.000000,2.000000,0.000000,0.000000,28.000000,9.000000,195.000000,5.000000,2.000000,6.000000,6.000000,5.000000,24.000000,24.000000,1.000000,23.000000,22.000000,25.000000,5.000000,6.000000,5.000000,5.000000,36.000000,1.000000,29.000000,1.000000,23.000000,5.000000,0.000000,0.000000,0.000000
75%,1.000000,8.000000,0.000000,8.000000,35.000000,8.000000,4.000000,0.000000,4.000000,0.000000,0.000000,30.000000,10.000000,316.000000,8.000000,2.000000,8.000000,8.000000,5.000000,26.000000,29.000000,2.000000,27.000000,24.000000,28.000000,6.000000,9.000000,8.000000,8.000000,42.000000,3.000000,34.000000,3.000000,28.000000,8.000000,0.000000,0.000000,0.000000
max,6.000000,11.000000,4.000000,11.000000,45.000000,19.000000,5.000000,5.000000,5.000000,3.000000,3.000000,35.000000,15.000000,369.000000,11.000000,4.000000,11.000000,12.000000,11.000000,32.000000,39.000000,3.000000,35.000000,26.000000,34.000000,10.000000,11.000000,11.000000,11.000000,60.000000,5.000000,40.000000,5.000000,36.000000,14.000000,3.000000,5.000000,3.000000


In [21]:
ord_in_dict = defaultdict(LabelEncoder)
for i in ord_in_master_dataset.columns:
    ord_in_master_dataset[i].fillna(-1, inplace=True)
    ord_in_master_dataset[i] = ord_in_master_dataset[i].astype("category")
#     ord_in_dict[i] = LabelEncoder()
#     ord_in_master_dataset[i] = ord_in_dict[i].fit_transform(ord_in_master_dataset[i])

In [22]:
ord_in_master_dataset.describe(include="all")

,o_0120,o_0125,o_0129,o_0132,o_0141,o_0144,o_0147,o_0152,o_0153,o_0154,o_0157,o_0168,o_0175,o_0176,o_0179,o_0201,o_0202,o_0208,o_0217,o_0221,o_0223,o_0230,o_0231,o_0241,o_0248,o_0264,o_0265,o_0268,o_0270,o_0274,o_0276,o_0279,o_0285,o_0286,o_0301,o_0314,o_0315,o_0323
count,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0,14644.0
unique,8.0,13.0,6.0,13.0,47.0,21.0,7.0,7.0,7.0,5.0,5.0,37.0,17.0,367,13.0,6.0,13.0,14.0,13.0,34.0,41.0,5.0,37.0,28.0,36.0,11,13.0,13.0,13.0,59.0,7.0,42.0,7.0,38.0,16.0,5.0,7.0,5.0
top,0.0,-1.0,-1.0,-1.0,-1.0,7.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,369,-1.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,4,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
freq,10157.0,7273.0,11620.0,9780.0,5048.0,3947.0,9417.0,11470.0,2986.0,11469.0,11468.0,7509.0,6214.0,1544,11171.0,5576.0,9862.0,11640.0,7158.0,10514.0,7125.0,4765.0,11619.0,7191.0,11631.0,3888,8065.0,7642.0,7031.0,9833.0,11415.0,6161.0,10827.0,9728.0,5508.0,11468.0,7299.0,11469.0


## Building Model